# Renewable Energy Prediction
Data from: https://data.open-power-system-data.org/

In [796]:
#Importing libraries for visualization, graphs, and modeling
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns
sns.set()
from pylab import *
from scipy import special, optimize
import sys
import os
pd.set_option('display.max_columns', None)
from fbprophet import Prophet

In [797]:
filepath_prod = "/Users/robertprior/Desktop/ML Projects/Final Project/time_series_60min_singleindex.csv"
#filepath_w = "/⁨Users⁩/⁨robertprior⁩/Desktop⁩/⁨ML Projects⁩/Final Project⁩/weather_data.csv"
filepath_w = "/Users/robertprior/Desktop/ML Projects/Final Project/weather_data.csv"


In [798]:
prod = pd.read_csv(filepath_prod,
                        usecols=(lambda s: s.startswith('utc') | s.startswith('DE')),
                        parse_dates=[0], index_col=0)

In [799]:
weather = pd.read_csv(filepath_w, parse_dates=[0], index_col=0)

Electricity production first

In [800]:
#prod.info()

In [801]:
prod_missing = prod.isnull().columns
prod_missing = prod_missing.drop(['DE_wind_generation_actual','DE_solar_generation_actual'])
print(prod_missing)

Index(['DE_load_entsoe_power_statistics', 'DE_load_entsoe_transparency',
       'DE_price_day_ahead', 'DE_solar_capacity', 'DE_solar_profile',
       'DE_wind_capacity', 'DE_wind_profile', 'DE_wind_offshore_capacity',
       'DE_wind_offshore_generation_actual', 'DE_wind_offshore_profile',
       'DE_wind_onshore_capacity', 'DE_wind_onshore_generation_actual',
       'DE_wind_onshore_profile', 'DE_50hertz_solar_generation_actual',
       'DE_50hertz_solar_generation_forecast',
       'DE_50hertz_wind_generation_actual',
       'DE_50hertz_wind_generation_forecast',
       'DE_50hertz_wind_offshore_generation_actual',
       'DE_50hertz_wind_offshore_generation_forecast',
       'DE_50hertz_wind_onshore_generation_actual',
       'DE_50hertz_wind_onshore_generation_forecast',
       'DE_AT_LU_price_day_ahead', 'DE_amprion_solar_generation_actual',
       'DE_amprion_solar_generation_forecast',
       'DE_amprion_wind_generation_actual',
       'DE_amprion_wind_generation_forecast',
    

In [802]:
prod.isnull().sum()

DE_load_entsoe_power_statistics                  1466
DE_load_entsoe_transparency                     78145
DE_price_day_ahead                                  0
DE_solar_capacity                                3624
DE_solar_generation_actual                      54743
DE_solar_profile                                56254
DE_wind_capacity                                35087
DE_wind_generation_actual                       37185
DE_wind_profile                                 38696
DE_wind_offshore_capacity                       35087
DE_wind_offshore_generation_actual              87650
DE_wind_offshore_profile                        91272
DE_wind_onshore_capacity                         3624
DE_wind_onshore_generation_actual               89761
DE_wind_onshore_profile                         91272
DE_50hertz_solar_generation_actual              52584
DE_50hertz_solar_generation_forecast            52584
DE_50hertz_wind_generation_actual                   4
DE_50hertz_wind_generation_f

In [803]:
#prod = prod[pd.notnull(prod['DE_solar_generation_actual'])]

In [804]:
prod[prod_missing] = prod[prod_missing].apply(lambda x: x.fillna(x.mean()), axis=1)

In [805]:
prod.isnull().sum()

DE_load_entsoe_power_statistics                     0
DE_load_entsoe_transparency                         0
DE_price_day_ahead                                  0
DE_solar_capacity                                   0
DE_solar_generation_actual                      54743
DE_solar_profile                                    0
DE_wind_capacity                                    0
DE_wind_generation_actual                       37185
DE_wind_profile                                     0
DE_wind_offshore_capacity                           0
DE_wind_offshore_generation_actual                  0
DE_wind_offshore_profile                            0
DE_wind_onshore_capacity                            0
DE_wind_onshore_generation_actual                   0
DE_wind_onshore_profile                             0
DE_50hertz_solar_generation_actual                  0
DE_50hertz_solar_generation_forecast                0
DE_50hertz_wind_generation_actual                   0
DE_50hertz_wind_generation_f

In [806]:
prod = prod.dropna()

In [807]:
prod.isnull().sum()

DE_load_entsoe_power_statistics                 0
DE_load_entsoe_transparency                     0
DE_price_day_ahead                              0
DE_solar_capacity                               0
DE_solar_generation_actual                      0
DE_solar_profile                                0
DE_wind_capacity                                0
DE_wind_generation_actual                       0
DE_wind_profile                                 0
DE_wind_offshore_capacity                       0
DE_wind_offshore_generation_actual              0
DE_wind_offshore_profile                        0
DE_wind_onshore_capacity                        0
DE_wind_onshore_generation_actual               0
DE_wind_onshore_profile                         0
DE_50hertz_solar_generation_actual              0
DE_50hertz_solar_generation_forecast            0
DE_50hertz_wind_generation_actual               0
DE_50hertz_wind_generation_forecast             0
DE_50hertz_wind_offshore_generation_actual      0


In [808]:
target_wind = prod['DE_wind_generation_actual']
target_solar = prod['DE_solar_generation_actual']

In [809]:
prod.head()

,DE_load_entsoe_power_statistics,DE_load_entsoe_transparency,DE_price_day_ahead,DE_solar_capacity,DE_solar_generation_actual,DE_solar_profile,DE_wind_capacity,DE_wind_generation_actual,DE_wind_profile,DE_wind_offshore_capacity,DE_wind_offshore_generation_actual,DE_wind_offshore_profile,DE_wind_onshore_capacity,DE_wind_onshore_generation_actual,DE_wind_onshore_profile,DE_50hertz_solar_generation_actual,DE_50hertz_solar_generation_forecast,DE_50hertz_wind_generation_actual,DE_50hertz_wind_generation_forecast,DE_50hertz_wind_offshore_generation_actual,DE_50hertz_wind_offshore_generation_forecast,DE_50hertz_wind_onshore_generation_actual,DE_50hertz_wind_onshore_generation_forecast,DE_AT_LU_price_day_ahead,DE_amprion_solar_generation_actual,DE_amprion_solar_generation_forecast,DE_amprion_wind_generation_actual,DE_amprion_wind_generation_forecast,DE_amprion_wind_onshore_generation_actual,DE_tennet_solar_generation_actual,DE_tennet_solar_generation_forecast,DE_tennet_wind_generation_actual,DE_tennet_wind_generation_forecast,DE_tennet_wind_offshore_generation_actual,DE_tennet_wind_onshore_generation_actual,DE_transnetbw_solar_generation_actual,DE_transnetbw_solar_generation_forecast,DE_transnetbw_wind_generation_actual,DE_transnetbw_wind_generation_forecast,DE_transnetbw_wind_onshore_generation_actual
utc_timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2011-12-31 23:00:00+00:00,40318.0,4764.617446,25.12,25969.0,0.0,0.0,27153.0,4576.0,0.1685,188.0,4764.617446,4764.617446,26965.0,4764.617446,4764.617446,0.0,0.0,1272.0,1745.0,4764.617446,4764.617446,4764.617446,4764.617446,4764.617446,0.0,0.0,1542.0,1257.0,1542.0,0.0,0.0,1583.0,1822.0,95.0,1488.0,0.0,0.0,179.0,87.0,179.0
2012-01-01 00:00:00+00:00,38406.0,4723.029264,23.64,25969.0,0.0,0.0,27153.0,4871.0,0.1794,188.0,4723.029264,4723.029264,26965.0,4723.029264,4723.029264,0.0,0.0,1359.0,1793.0,4723.029264,4723.029264,4723.029264,4723.029264,4723.029264,0.0,0.0,1548.0,1219.0,1548.0,0.0,0.0,1816.0,2076.0,101.0,1716.0,0.0,0.0,148.0,68.0,148.0
2012-01-01 01:00:00+00:00,36501.0,4711.793900,19.03,25969.0,0.0,0.0,27153.0,5409.0,0.1992,188.0,4711.793900,4711.793900,26965.0,4711.793900,4711.793900,0.0,0.0,1339.0,1874.0,4711.793900,4711.793900,4711.793900,4711.793900,4711.793900,0.0,0.0,1650.0,1237.0,1650.0,0.0,0.0,2312.0,2472.0,100.0,2212.0,0.0,0.0,108.0,73.0,108.0
2012-01-01 02:00:00+00:00,35305.0,4735.864086,15.97,25969.0,0.0,0.0,27153.0,6094.0,0.2244,188.0,4735.864086,4735.864086,26965.0,4735.864086,4735.864086,0.0,0.0,1436.0,1976.0,4735.864086,4735.864086,4735.864086,4735.864086,4735.864086,0.0,0.0,1642.0,1254.0,1642.0,0.0,0.0,2882.0,2945.0,100.0,2781.0,0.0,0.0,134.0,82.0,134.0
2012-01-01 03:00:00+00:00,34568.0,4766.933857,15.90,25969.0,0.0,0.0,27153.0,6734.0,0.2480,188.0,4766.933857,4766.933857,26965.0,4766.933857,4766.933857,0.0,0.0,1708.0,2113.0,4766.933857,4766.933857,4766.933857,4766.933857,4766.933857,0.0,0.0,1710.0,1282.0,1710.0,0.0,0.0,3162.0,3376.0,96.0,3066.0,0.0,0.0,154.0,84.0,154.0


In [810]:
len(target_solar)

54070

In [811]:
target_solar.tail()

utc_timestamp
2018-03-05 18:00:00+00:00    0.0
2018-03-05 19:00:00+00:00    0.0
2018-03-05 20:00:00+00:00    0.0
2018-03-05 21:00:00+00:00    0.0
2018-03-05 22:00:00+00:00    0.0
Name: DE_solar_generation_actual, dtype: float64

### Weather

In [812]:
weather.tail(10)

,AT_windspeed_10m,AT_temperature,AT_radiation_direct_horizontal,AT_radiation_diffuse_horizontal,BE_windspeed_10m,BE_temperature,BE_radiation_direct_horizontal,BE_radiation_diffuse_horizontal,CH_windspeed_10m,CH_temperature,CH_radiation_direct_horizontal,CH_radiation_diffuse_horizontal,CZ_windspeed_10m,CZ_temperature,CZ_radiation_direct_horizontal,CZ_radiation_diffuse_horizontal,DE_windspeed_10m,DE_temperature,DE_radiation_direct_horizontal,DE_radiation_diffuse_horizontal,DE11_windspeed_10m,DE11_temperature,DE11_radiation_direct_horizontal,DE11_radiation_diffuse_horizontal,DE12_windspeed_10m,DE12_temperature,DE12_radiation_direct_horizontal,DE12_radiation_diffuse_horizontal,DE13_windspeed_10m,DE13_temperature,DE13_radiation_direct_horizontal,DE13_radiation_diffuse_horizontal,DE14_windspeed_10m,DE14_temperature,DE14_radiation_direct_horizontal,DE14_radiation_diffuse_horizontal,DE21_windspeed_10m,DE21_temperature,DE21_radiation_direct_horizontal,DE21_radiation_diffuse_horizontal,DE22_windspeed_10m,DE22_temperature,DE22_radiation_direct_horizontal,DE22_radiation_diffuse_horizontal,DE23_windspeed_10m,DE23_temperature,DE23_radiation_direct_horizontal,DE23_radiation_diffuse_horizontal,DE24_windspeed_10m,DE24_temperature,DE24_radiation_direct_horizontal,DE24_radiation_diffuse_horizontal,DE25_windspeed_10m,DE25_temperature,DE25_radiation_direct_horizontal,DE25_radiation_diffuse_horizontal,DE26_windspeed_10m,DE26_temperature,DE26_radiation_direct_horizontal,DE26_radiation_diffuse_horizontal,DE27_windspeed_10m,DE27_temperature,DE27_radiation_direct_horizontal,DE27_radiation_diffuse_horizontal,DE30_windspeed_10m,DE30_temperature,DE30_radiation_direct_horizontal,DE30_radiation_diffuse_horizontal,DE40_windspeed_10m,DE40_temperature,DE40_radiation_direct_horizontal,DE40_radiation_diffuse_horizontal,DE50_windspeed_10m,DE50_temperature,DE50_radiation_direct_horizontal,DE50_radiation_diffuse_horizontal,DE60_windspeed_10m,DE60_temperature,DE60_radiation_direct_horizontal,DE60_radiation_diffuse_horizontal,DE71_windspeed_10m,DE71_temperature,DE71_radiation_direct_horizontal,DE71_radiation_diffuse_horizontal,DE72_windspeed_10m,DE72_temperature,DE72_radiation_direct_horizontal,DE72_radiation_diffuse_horizontal,DE73_windspeed_10m,DE73_temperature,DE73_radiation_direct_horizontal,DE73_radiation_diffuse_horizontal,DE80_windspeed_10m,DE80_temperature,DE80_radiation_direct_horizontal,DE80_radiation_diffuse_horizontal,DE91_windspeed_10m,DE91_temperature,DE91_radiation_direct_horizontal,DE91_radiation_diffuse_horizontal,DE92_windspeed_10m,DE92_temperature,DE92_radiation_direct_horizontal,DE92_radiation_diffuse_horizontal,DE93_windspeed_10m,DE93_temperature,DE93_radiation_direct_horizontal,DE93_radiation_diffuse_horizontal,DE94_windspeed_10m,DE94_temperature,DE94_radiation_direct_horizontal,DE94_radiation_diffuse_horizontal,DEA1_windspeed_10m,DEA1_temperature,DEA1_radiation_direct_horizontal,DEA1_radiation_diffuse_horizontal,DEA2_windspeed_10m,DEA2_temperature,DEA2_radiation_direct_horizontal,DEA2_radiation_diffuse_horizontal,DEA3_windspeed_10m,DEA3_temperature,DEA3_radiation_direct_horizontal,DEA3_radiation_diffuse_horizontal,DEA4_windspeed_10m,DEA4_temperature,DEA4_radiation_direct_horizontal,DEA4_radiation_diffuse_horizontal,DEA5_windspeed_10m,DEA5_temperature,DEA5_radiation_direct_horizontal,DEA5_radiation_diffuse_horizontal,DEB1_windspeed_10m,DEB1_temperature,DEB1_radiation_direct_horizontal,DEB1_radiation_diffuse_horizontal,DEB2_windspeed_10m,DEB2_temperature,DEB2_radiation_direct_horizontal,DEB2_radiation_diffuse_horizontal,DEB3_windspeed_10m,DEB3_temperature,DEB3_radiation_direct_horizontal,DEB3_radiation_diffuse_horizontal,DEC0_windspeed_10m,DEC0_temperature,DEC0_radiation_direct_horizontal,DEC0_radiation_diffuse_horizontal,DED2_windspeed_10m,DED2_temperature,DED2_radiation_direct_horizontal,DED2_radiation_diffuse_horizontal,DED4_windspeed_10m,DED4_temperature,DED4_radiation_direct_horizontal,DED4_radiation_diffuse_horizontal,DED5_windspeed_10m,DED

look at merging the phrames but only weather data in our target area

In [813]:
weather.isnull().sum()[1].sum()

0

So now missing values in our weather dataset, lets merge

In [814]:
combined = prod.merge(weather, left_on=['utc_timestamp'], right_on=['utc_timestamp'], how='left')

In [815]:
combined.tail()

,DE_load_entsoe_power_statistics,DE_load_entsoe_transparency,DE_price_day_ahead,DE_solar_capacity,DE_solar_generation_actual,DE_solar_profile,DE_wind_capacity,DE_wind_generation_actual,DE_wind_profile,DE_wind_offshore_capacity,DE_wind_offshore_generation_actual,DE_wind_offshore_profile,DE_wind_onshore_capacity,DE_wind_onshore_generation_actual,DE_wind_onshore_profile,DE_50hertz_solar_generation_actual,DE_50hertz_solar_generation_forecast,DE_50hertz_wind_generation_actual,DE_50hertz_wind_generation_forecast,DE_50hertz_wind_offshore_generation_actual,DE_50hertz_wind_offshore_generation_forecast,DE_50hertz_wind_onshore_generation_actual,DE_50hertz_wind_onshore_generation_forecast,DE_AT_LU_price_day_ahead,DE_amprion_solar_generation_actual,DE_amprion_solar_generation_forecast,DE_amprion_wind_generation_actual,DE_amprion_wind_generation_forecast,DE_amprion_wind_onshore_generation_actual,DE_tennet_solar_generation_actual,DE_tennet_solar_generation_forecast,DE_tennet_wind_generation_actual,DE_tennet_wind_generation_forecast,DE_tennet_wind_offshore_generation_actual,DE_tennet_wind_onshore_generation_actual,DE_transnetbw_solar_generation_actual,DE_transnetbw_solar_generation_forecast,DE_transnetbw_wind_generation_actual,DE_transnetbw_wind_generation_forecast,DE_transnetbw_wind_onshore_generation_actual,AT_windspeed_10m,AT_temperature,AT_radiation_direct_horizontal,AT_radiation_diffuse_horizontal,BE_windspeed_10m,BE_temperature,BE_radiation_direct_horizontal,BE_radiation_diffuse_horizontal,CH_windspeed_10m,CH_temperature,CH_radiation_direct_horizontal,CH_radiation_diffuse_horizontal,CZ_windspeed_10m,CZ_temperature,CZ_radiation_direct_horizontal,CZ_radiation_diffuse_horizontal,DE_windspeed_10m,DE_temperature,DE_radiation_direct_horizontal,DE_radiation_diffuse_horizontal,DE11_windspeed_10m,DE11_temperature,DE11_radiation_direct_horizontal,DE11_radiation_diffuse_horizontal,DE12_windspeed_10m,DE12_temperature,DE12_radiation_direct_horizontal,DE12_radiation_diffuse_horizontal,DE13_windspeed_10m,DE13_temperature,DE13_radiation_direct_horizontal,DE13_radiation_diffuse_horizontal,DE14_windspeed_10m,DE14_temperature,DE14_radiation_direct_horizontal,DE14_radiation_diffuse_horizontal,DE21_windspeed_10m,DE21_temperature,DE21_radiation_direct_horizontal,DE21_radiation_diffuse_horizontal,DE22_windspeed_10m,DE22_temperature,DE22_radiation_direct_horizontal,DE22_radiation_diffuse_horizontal,DE23_windspeed_10m,DE23_temperature,DE23_radiation_direct_horizontal,DE23_radiation_diffuse_horizontal,DE24_windspeed_10m,DE24_temperature,DE24_radiation_direct_horizontal,DE24_radiation_diffuse_horizontal,DE25_windspeed_10m,DE25_temperature,DE25_radiation_direct_horizontal,DE25_radiation_diffuse_horizontal,DE26_windspeed_10m,DE26_temperature,DE26_radiation_direct_horizontal,DE26_radiation_diffuse_horizontal,DE27_windspeed_10m,DE27_temperature,DE27_radiation_direct_horizontal,DE27_radiation_diffuse_horizontal,DE30_windspeed_10m,DE30_temperature,DE30_radiation_direct_horizontal,DE30_radiation_diffuse_horizontal,DE40_windspeed_10m,DE40_temperature,DE40_radiation_direct_horizontal,DE40_radiation_diffuse_horizontal,DE50_windspeed_10m,DE50_temperature,DE50_radiation_direct_horizontal,DE50_radiation_diffuse_horizontal,DE60_windspeed_10m,DE60_temperature,DE60_radiation_direct_horizontal,DE60_radiation_diffuse_horizontal,DE71_windspeed_10m,DE71_temperature,DE71_radiation_direct_horizontal,DE71_radiation_diffuse_horizontal,DE72_windspeed_10m,DE72_temperature,DE72_radiation_direct_horizontal,DE72_radiation_diffuse_horizontal,DE73_windspeed_10m,DE73_temperature,DE73_radiation_direct_horizontal,DE73_radiation_diffuse_horizontal,DE80_windspeed_10m,DE80_temperature,DE80_radiation_direct_horizontal,DE80_radiation_diffuse_horizontal,DE91_windspeed_10m,DE91_temperature,DE91_radiation_direct_horizontal,DE91_radiation_diffuse_horizontal,DE92_windspeed_10m,DE92_temperature,DE92_radiation_direct_horizontal,DE92_radiation_diffuse_horizontal,DE93_windspeed_10m,DE93_temperature,DE93_radiat

In [816]:
combined.isnull().sum()

DE_load_entsoe_power_statistics                     0
DE_load_entsoe_transparency                         0
DE_price_day_ahead                                  0
DE_solar_capacity                                   0
DE_solar_generation_actual                          0
DE_solar_profile                                    0
DE_wind_capacity                                    0
DE_wind_generation_actual                           0
DE_wind_profile                                     0
DE_wind_offshore_capacity                           0
DE_wind_offshore_generation_actual                  0
DE_wind_offshore_profile                            0
DE_wind_onshore_capacity                            0
DE_wind_onshore_generation_actual                   0
DE_wind_onshore_profile                             0
DE_50hertz_solar_generation_actual                  0
DE_50hertz_solar_generation_forecast                0
DE_50hertz_wind_generation_actual                   0
DE_50hertz_wind_generation_f

In [817]:
combined = combined.dropna()

In [818]:
combined.tail()

,DE_load_entsoe_power_statistics,DE_load_entsoe_transparency,DE_price_day_ahead,DE_solar_capacity,DE_solar_generation_actual,DE_solar_profile,DE_wind_capacity,DE_wind_generation_actual,DE_wind_profile,DE_wind_offshore_capacity,DE_wind_offshore_generation_actual,DE_wind_offshore_profile,DE_wind_onshore_capacity,DE_wind_onshore_generation_actual,DE_wind_onshore_profile,DE_50hertz_solar_generation_actual,DE_50hertz_solar_generation_forecast,DE_50hertz_wind_generation_actual,DE_50hertz_wind_generation_forecast,DE_50hertz_wind_offshore_generation_actual,DE_50hertz_wind_offshore_generation_forecast,DE_50hertz_wind_onshore_generation_actual,DE_50hertz_wind_onshore_generation_forecast,DE_AT_LU_price_day_ahead,DE_amprion_solar_generation_actual,DE_amprion_solar_generation_forecast,DE_amprion_wind_generation_actual,DE_amprion_wind_generation_forecast,DE_amprion_wind_onshore_generation_actual,DE_tennet_solar_generation_actual,DE_tennet_solar_generation_forecast,DE_tennet_wind_generation_actual,DE_tennet_wind_generation_forecast,DE_tennet_wind_offshore_generation_actual,DE_tennet_wind_onshore_generation_actual,DE_transnetbw_solar_generation_actual,DE_transnetbw_solar_generation_forecast,DE_transnetbw_wind_generation_actual,DE_transnetbw_wind_generation_forecast,DE_transnetbw_wind_onshore_generation_actual,AT_windspeed_10m,AT_temperature,AT_radiation_direct_horizontal,AT_radiation_diffuse_horizontal,BE_windspeed_10m,BE_temperature,BE_radiation_direct_horizontal,BE_radiation_diffuse_horizontal,CH_windspeed_10m,CH_temperature,CH_radiation_direct_horizontal,CH_radiation_diffuse_horizontal,CZ_windspeed_10m,CZ_temperature,CZ_radiation_direct_horizontal,CZ_radiation_diffuse_horizontal,DE_windspeed_10m,DE_temperature,DE_radiation_direct_horizontal,DE_radiation_diffuse_horizontal,DE11_windspeed_10m,DE11_temperature,DE11_radiation_direct_horizontal,DE11_radiation_diffuse_horizontal,DE12_windspeed_10m,DE12_temperature,DE12_radiation_direct_horizontal,DE12_radiation_diffuse_horizontal,DE13_windspeed_10m,DE13_temperature,DE13_radiation_direct_horizontal,DE13_radiation_diffuse_horizontal,DE14_windspeed_10m,DE14_temperature,DE14_radiation_direct_horizontal,DE14_radiation_diffuse_horizontal,DE21_windspeed_10m,DE21_temperature,DE21_radiation_direct_horizontal,DE21_radiation_diffuse_horizontal,DE22_windspeed_10m,DE22_temperature,DE22_radiation_direct_horizontal,DE22_radiation_diffuse_horizontal,DE23_windspeed_10m,DE23_temperature,DE23_radiation_direct_horizontal,DE23_radiation_diffuse_horizontal,DE24_windspeed_10m,DE24_temperature,DE24_radiation_direct_horizontal,DE24_radiation_diffuse_horizontal,DE25_windspeed_10m,DE25_temperature,DE25_radiation_direct_horizontal,DE25_radiation_diffuse_horizontal,DE26_windspeed_10m,DE26_temperature,DE26_radiation_direct_horizontal,DE26_radiation_diffuse_horizontal,DE27_windspeed_10m,DE27_temperature,DE27_radiation_direct_horizontal,DE27_radiation_diffuse_horizontal,DE30_windspeed_10m,DE30_temperature,DE30_radiation_direct_horizontal,DE30_radiation_diffuse_horizontal,DE40_windspeed_10m,DE40_temperature,DE40_radiation_direct_horizontal,DE40_radiation_diffuse_horizontal,DE50_windspeed_10m,DE50_temperature,DE50_radiation_direct_horizontal,DE50_radiation_diffuse_horizontal,DE60_windspeed_10m,DE60_temperature,DE60_radiation_direct_horizontal,DE60_radiation_diffuse_horizontal,DE71_windspeed_10m,DE71_temperature,DE71_radiation_direct_horizontal,DE71_radiation_diffuse_horizontal,DE72_windspeed_10m,DE72_temperature,DE72_radiation_direct_horizontal,DE72_radiation_diffuse_horizontal,DE73_windspeed_10m,DE73_temperature,DE73_radiation_direct_horizontal,DE73_radiation_diffuse_horizontal,DE80_windspeed_10m,DE80_temperature,DE80_radiation_direct_horizontal,DE80_radiation_diffuse_horizontal,DE91_windspeed_10m,DE91_temperature,DE91_radiation_direct_horizontal,DE91_radiation_diffuse_horizontal,DE92_windspeed_10m,DE92_temperature,DE92_radiation_direct_horizontal,DE92_radiation_diffuse_horizontal,DE93_windspeed_10m,DE93_temperature,DE93_radiat

## Modeling

In [ ]:
m = Prophet()

In [ ]:
solar_prophet = []
solar_prophet = combined['DE_solar_generation_actual'].copy()
solar_prophet = pd.DataFrame(solar_prophet)
solar_prophet['ds'] = solar_prophet.index

In [ ]:
solar_prophet.info()

In [ ]:
solar_prophet.columns = ['y', 'ds']
#solar_prophet.ds = pd.to_datetime(solar_prophet.ds, utc=True, errors='coerce')
#solar_prophet['ds'] = solar_prophet['ds'].dt.tz_localize('UTC')
solar_prophet['ds'] = solar_prophet['ds'].dt.tz_convert(None)
#solar_prophet['ds'] = pd.to_datetime(solar_prophet['ds'], utc=True, format=timestamp_format)

In [ ]:
solar_prophet = solar_prophet.reset_index(drop=True)


In [ ]:
solar_prophet.tail(20)

In [ ]:
solar_prophet['floor'] = 0
solar_prophet['cap'] = 50000

In [ ]:
#used bbeofre hourly seasonality added
#m = Prophet(changepoint_prior_scale=0.01, growth='logistic', yearly_seasonality=20).fit(solar_prophet)

In [ ]:
m = Prophet(changepoint_prior_scale=0.01, growth='logistic', yearly_seasonality=20, seasonality_mode='multiplicative')
m.add_seasonality(name='hourly', period=24, fourier_order=7)
m.fit(solar_prophet)

In [ ]:
future = m.make_future_dataframe(periods=24, freq='H')
future['floor'] = 0
future['cap'] = 50000

In [ ]:
fcst = m.predict(future)

In [ ]:
fcst.tail(10)

In [ ]:
fig = m.plot(fcst)

In [ ]:
fig = m.plot_components(fcst)


### Wind

In [ ]:
wind_prophet = []
wind_prophet = combined['DE_wind_generation_actual'].copy()

In [ ]:
wind_prophet = pd.DataFrame(wind_prophet)

In [ ]:
wind_prophet['ds'] = wind_prophet.index

In [ ]:
wind_prophet.columns = ['y', 'ds']
#solar_prophet.ds = pd.to_datetime(solar_prophet.ds, utc=True, errors='coerce')
#solar_prophet['ds'] = solar_prophet['ds'].dt.tz_localize('UTC')
wind_prophet['ds'] = wind_prophet['ds'].dt.tz_convert(None)

In [ ]:
wind_prophet = wind_prophet.reset_index(drop=True)

In [ ]:
wind_prophet['y']=wind_prophet.y.mask(wind_prophet.y == 0, 0.0001)

In [ ]:
#solar_prophet_log = solar_prophet.copy()
#solar_prophet_log['y'] = np.log(solar_prophet['y'])

In [ ]:
#solar_prophet_log.head()

In [ ]:
wind_prophet['floor'] = 0
wind_prophet['cap'] = 50000

In [ ]:
m = Prophet(changepoint_prior_scale=0.0001, growth='logistic', yearly_seasonality=20, seasonality_mode='multiplicative')
m.add_seasonality(name='hourly', period=24, fourier_order=5)
m.fit(wind_prophet)

In [ ]:
future = m.make_future_dataframe(periods=24, freq='H')
future['floor'] = 0
future['cap'] = 50

In [ ]:
fcst_wind = m.predict(future)

In [ ]:

#fcst_columns = fcst.columns.values
#fcst_ds = fcst['ds']
#fcst_ds = pd.DataFrame(fcst_ds)
#fcst = fcst.drop['ds']
#fcst_columns = fcst_columns.tolist()
#fcst_columns.remove('ds')

In [ ]:
#fcst = np.exp(fcst[fcst_columns])
#fcst = pd.DataFrame(fcst)
#fcst = fcst_ds.join(fcst)

In [ ]:
fcst_wind.head()

In [ ]:
fig = m.plot(fcst_wind)

In [ ]:
fig = m.plot_components(fcst_wind)

# Daily

## Wind

In [ ]:
# change confidence interval bandwidth with 
interval_width=0.95

In [ ]:
daily_w =  combined['DE_wind_generation_actual'].resample('D').sum()

In [ ]:
#daily['DE_wind_generation_actual'].rolling(30, center=True).sum().plot(style=[':', '--', '-'])
daily_w.rolling(30, center=True).sum().plot(style=[':', '--', '-'])

In [ ]:
wind_prophet = []
wind_prophet = daily_w.copy()

In [ ]:
daily.head()

In [ ]:
wind_prophet = pd.DataFrame(wind_prophet)

In [ ]:
wind_prophet.head()

In [ ]:
wind_prophet['ds'] = wind_prophet.index

In [ ]:
wind_prophet.columns = ['y', 'ds']
wind_prophet['ds'] = wind_prophet['ds'].dt.tz_convert(None)

In [ ]:
wind_prophet = wind_prophet.reset_index(drop=True)

In [ ]:
#wind_prophet['y']=wind_prophet.y.mask(wind_prophet.y == 0, 0.0001)

In [ ]:
wind_prophet['floor'] = 0
wind_prophet['cap'] = 50000

In [ ]:
m_wind = Prophet(changepoint_prior_scale=0.05, interval_width=0.95)
m_wind.fit(wind_prophet)

In [ ]:
future = m_wind.make_future_dataframe(periods=30*2, freq='D')
future['floor'] = 0
future['cap'] = 50

In [ ]:
fcst_wind = m_wind.predict(future)

In [ ]:
fig = m_wind.plot(fcst_wind)

In [ ]:
fig = m_wind.plot_components(fcst_wind)

## Solar

In [ ]:
daily_s =  combined['DE_solar_generation_actual'].resample('D').sum()

In [ ]:
daily_s.rolling(30, center=True).sum().plot(style=[':', '--', '-'])

In [ ]:
solar_prophet = []
solar_prophet = daily_s.copy()

solar_prophet = pd.DataFrame(solar_prophet)
solar_prophet['ds'] = solar_prophet.index

In [ ]:

solar_prophet.columns = ['y', 'ds']
solar_prophet['ds'] = solar_prophet['ds'].dt.tz_convert(None)
solar_prophet = solar_prophet.reset_index(drop=True)

In [ ]:
solar_prophet['floor'] = 0
solar_prophet['cap'] = 500000

In [ ]:
m_solar = Prophet(changepoint_prior_scale=0.05, interval_width=0.95)
m_solar.fit(solar_prophet)

In [ ]:
future = m_solar.make_future_dataframe(periods=30*2, freq='D')
future['floor'] = 0
future['cap'] = 50

In [ ]:
fcst_solar = m_solar.predict(future)

In [ ]:
fig = m_solar.plot(fcst_solar)

In [ ]:
fig = m_solar.plot_components(fcst_solar)

## Combining Solar and Wind

In [ ]:
solar_names = ['solar_%s' % column for column in fcst_solar.columns]

In [ ]:

wind_names = ['wind_%s' % column for column in fcst_wind.columns]
merge_solar_forecast = fcst_solar.copy()
merge_wind_forecast = fcst_wind.copy()
merge_solar_forecast.columns = solar_names
merge_wind_forecast.columns = wind_names
forecast = pd.merge(merge_solar_forecast, merge_wind_forecast, how = 'inner', left_on = 'solar_ds', right_on = 'wind_ds')
forecast = forecast.rename(columns={'solar_ds': 'Date'}).drop('wind_ds', axis=1)
forecast.head()

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(forecast['Date'], forecast['solar_trend'], 'b-')
plt.plot(forecast['Date'], forecast['wind_trend'], 'r-')
plt.legend(); plt.xlabel('Date'); plt.ylabel('Electricity Production')
plt.title('Solar vs. Wind Electricity Production Trend');

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(forecast['Date'], forecast['solar_yhat'], 'b-')
plt.plot(forecast['Date'], forecast['wind_yhat'], 'r-')
plt.legend(); plt.xlabel('Date'); plt.ylabel('Electricity Production')
plt.title('Solar vs. Wind Electricity Production Estimate');

In [ ]:
m_solar.plot_components(fcst_solar);

In [ ]:
m_wind.plot_components(fcst_wind);

# Cross validation for time series

In [ ]:
fig, ax = plt.subplots(10,1, figsize = (14, 20))
ax = ax.ravel()
j = 0
solar_prophet = []
solar_prophet = daily_s.copy()

solar_prophet = pd.DataFrame(solar_prophet)
solar_prophet['ds'] = solar_prophet.index

solar_prophet.columns = ['y', 'ds']
solar_prophet['ds'] = solar_prophet['ds'].dt.tz_convert(None)
solar_prophet = solar_prophet.reset_index(drop=True)

solar_prophet['floor'] = 0
solar_prophet['cap'] = 500000

for i in [0.01, 0.05, 0.10, 0.15, 0.20]:
    #RUT2 = RUT.reset_index()
    #RUT2 = RUT2.rename(columns = {'Date' : 'ds', 'Adj Close' : 'y'} )
    m_solar = Prophet(changepoint_prior_scale=i)
    #fit the prophet model on the data
    m.fit(solar_prophet)
    # make a dataframe of the next two years 
    future = m_solar.make_future_dataframe(periods=365)
    #predict on this future dataframe
    forecast = m_solar.predict(future)
    # plot the forecast 
    fig = m_solar.plot(forecast, ax = ax[j])
    ax[j].set_title('changepoint prior = ' + str(i))
    j += 1
    df_cv = cross_validation(m_solar, initial='365 days', period='180 days', horizon = '365 days')
    df_p = performance_metrics(df_cv)
    fig = plot_cross_validation_metric(df_cv, metric='mape', ax = ax[j])
    ax[j].set_title(str(str(i) + ' Change Point Prior Mean Absolute Percentage Error'))
    ax[j].set_ylim(0,0.5)
    j += 1